# Uppgift 2 - Sportstatistik
- Välj 2-4 sporter och skapa lämpliga grafer/diagram för att visualisera exempelvis:
- medaljfördelning mellan länder i sporterna, åldersfördelning i sporterna
- Skapa fler plots för att visualisera olika aspekter kring sporterna.

Skriv ner våra frågeställningar.
- Kolla igenom vilka sporter som varit med?
- Analysera "Tug-Of-War"! (dragkamp) Kul att tex kolla hur länge det varit med i OS, vilka har varit bäst? Osv.
- Finns det andra gamla sporter?
- Alpinism
- Aeronautics [https://www.topendsports.com/events/demonstration/aeronautics.htm](https://www.topendsports.com/events/demonstration/aeronautics.htm)
- Viktutvecklingen för diskuskastare och kulstötare över tid?
- 